# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_df = pd.read_csv('../WeatherPy/output_data/city_data.csv')
weather_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Urucará,-2.54,-57.76,100.92,29,77,6.87,BR,1603211716
1,Coari,-4.08,-63.14,92.84,47,83,3.09,BR,1603211716
2,Punta Arenas,-53.15,-70.92,48.20,45,75,21.92,CL,1603211344
3,Lavrentiya,65.58,-171.00,35.60,90,39,10.63,RU,1603211717
4,Saldanha,-33.01,17.94,64.40,52,0,17.22,ZA,1603211276


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configuring the gmap
gmaps.configure(api_key=g_key)

In [4]:
#store lat and lng as floats
locations = weather_data_df[["Lat", "Lng"]].astype(float)

#store humidity as float
humidity = weather_data_df['Humidity'].astype(float)

In [5]:
#creating the heatmap
fig = gmaps.figure(zoom_level = 2, center = (0,0))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
perf_conditions_df =  weather_data_df.loc[(weather_data_df['Max Temp'] < 80) & 
                                          (weather_data_df['Max Temp'] > 70) & 
                                          (weather_data_df['Wind Speed'] < 10) &  
                                          (weather_data_df['Cloudiness'] == 0)]
perf_conditions_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
219,Poum,-20.23,164.02,74.62,69,0,6.44,NC,1603211318
225,Ibrā’,22.69,58.53,70.03,32,0,1.57,OM,1603211766
248,Mutare,-18.97,32.67,73.89,29,0,4.05,ZW,1603211771
281,Umri,26.51,78.94,76.59,19,0,2.55,IN,1603211778
327,Sur,22.57,59.53,75.92,61,0,4.85,OM,1603211477
362,Dīnhāta,26.13,89.47,77.94,79,0,4.94,IN,1603211795
363,Orange Cove,36.62,-119.31,73.99,55,0,3.20,US,1603211796
365,Guerrero Negro,27.98,-114.06,77.13,62,0,6.71,MX,1603211796
406,Turaif,31.67,38.66,78.80,16,0,4.70,SA,1603211552
419,Balad,34.02,44.15,78.71,19,0,9.04,IQ,1603211812


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = perf_conditions_df.loc[:,['City','Country', 'Lat', 'Lng']]
hotel_df['Hotel Name'] = ' '
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
219,Poum,NC,-20.23,164.02,
225,Ibrā’,OM,22.69,58.53,
248,Mutare,ZW,-18.97,32.67,
281,Umri,IN,26.51,78.94,
327,Sur,OM,22.57,59.53,


In [8]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f'{lat}, {lng}'
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params).json()
    #pprint(name_address)
   # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = (name_address['results'][0]['name'])
    except:
        print('NOPE')
hotel_df

NOPE
NOPE


,City,Country,Lat,Lng,Hotel Name
219,Poum,NC,-20.23,164.02,Gîte kajeon
225,Ibrā’,OM,22.69,58.53,مزرعة راحة النفوس المنزفة
248,Mutare,ZW,-18.97,32.67,Ann Bruce Backpackers
281,Umri,IN,26.51,78.94,shri kheda pati bajrang bali ji mandir umri
327,Sur,OM,22.57,59.53,Sur Plaza Hotel
362,Dīnhāta,IN,26.13,89.47,Hotel Riya
363,Orange Cove,US,36.62,-119.31,
365,Guerrero Negro,MX,27.98,-114.06,"Casa Laguna, Bed & Breakfast"
406,Turaif,SA,31.67,38.66,Swiss Spirit Hotel & Suites Turaif
419,Balad,IQ,34.02,44.15,رضاوي زنوبة


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))